In [ ]:
import re, os, glob
import numpy as np
import matplotlib.pyplot as plt
import pandas
import seaborn as sns


In [ ]:
current_palette = sns.color_palette(palette='muted', n_colors=5)
sns.palplot(current_palette)
plt.show()
plt.clf()

current_palette = sns.color_palette(palette='bright', n_colors=5)
sns.palplot(current_palette)
plt.show()
plt.clf()

current_palette = sns.color_palette(palette='cubehelix', n_colors=4)
sns.palplot(current_palette)
plt.show()
plt.clf()
current_palette = sns.color_palette(palette='', n_colors=5)


In [ ]:
genes = pandas.read_excel('RBP_high_freq_mutations.xlsx', sheet_name='Genomic locations')
genes['pos'] = genes['Start']/1E6
genes.index = genes['Name']
print(genes)

In [ ]:
df = pandas.read_excel('RBP_high_freq_mutations.xlsx', sheet_name='Chromosome lengths')
print(df.head(2))

df['length'] = df['Total length (bp)']/1E6
total_len = df['length'].sum()

fig = plt.figure()
ax = plt.subplot(111, projection='polar')
print(list(current_palette.as_hex()))
colors = list(current_palette.as_hex()) # 'rbgk'
#colors = 'rbgk'
start_theta = 0
chrom_to_start_theta = {}
chrom_to_theta_per_million_bp = {}
chrom_to_color = {}

for n, chrom in enumerate(df.Chromosome):
    print(chrom)
    chr_len = float(df.loc[df['Chromosome']==chrom, 'length'])
    print(chr_len)
    
    chr_rad = 2 * np.pi * chr_len/total_len
    theta = np.arange(start_theta, chr_rad + start_theta, 0.01)
    r = [2 for x in theta]

    chrom_to_start_theta[chrom] = start_theta
    chrom_to_theta_per_million_bp[chrom] = chr_rad/chr_len
    chrom_to_color[chrom] = colors[n % len(colors)]
    print('st=', start_theta)
    ax.plot(theta, r, colors[n % len(colors)])
    ax.text(start_theta + chr_rad/2, 2.2, s=chrom, fontdict={'fontsize': 7})
    
    start_theta += chr_rad
    
for gene in genes['Name']:
    chrom = genes.loc[gene, 'Chromosome']
    pos = genes.loc[gene, 'pos']
    theta_pos = chrom_to_start_theta[chrom] + pos * chrom_to_theta_per_million_bp[chrom]
    print(f"{gene} {chrom} {pos} -> theta_pos = {theta_pos}")
    
    ax.plot([theta_pos] * 2, [0, 3], chrom_to_color[chrom], alpha=0.2)
    ax.text(theta_pos, 3, s=gene)
#ax.set_rmax(3)
#ax.set_rticks([])  # less radial ticks
#ax.set_xticks([])
#ax.set_xticklabels('')
ax.set_axis_off()

#ax.set_rlabel_position(-22.5)  # get radial labels away from plotted line
ax.grid(False)

#ax.set_title("A line plot on a polar axis", va='bottom')
fig.set_figwidth(5)
fig.set_figheight(5)
fig.savefig(fname="figs/chromosome_wheel.pdf")
plt.show()
plt.clf()
plt.close()